In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime, date


from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

from datetime import datetime, date
import csv

mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['figure.figsize'] = 4.8,3.2
mpl.rcParams['legend.frameon'] = True
mpl.rcParams['legend.fancybox'] = True


In [4]:
plot_spatialkde = False

# Read data

In [14]:
fulldf.iloc[10:15]

,year,date,time,dayofweek,location,WEATHER,PRIMARY_COLLISION_FACTOR,HIT_AND_RUN,PED_ACTION,ROAD_SURFACE,...,LIGHTING,ALCOHOL_INVOLVED,COUNT_PED_KILLED,COUNT_PED_INJURED,y,x,nhood,tractce10,police_district,hist_police_district
10,2003,2003-01-15,06:27:00,Wednesday,"UNION and STOCKTON, at intersection",Clear,Pedestrian right of way,Hit and run (felony),"Crossing, in crosswalk at intersection",Dry,...,Dark - Street lights,False,0,1,37.8005,-122.409,North Beach,010600,CENTRAL,CENTRAL
11,2003,2003-01-12,19:12:00,Sunday,"FOLSOM ST and 6TH ST, at intersection",Raining,Pedestrian right of way,Hit and run (felony),"Crossing, in crosswalk at intersection",Wet,...,Dark - Street lights,False,0,1,37.7785,-122.406,South of Market,017802,SOUTHERN,SOUTHERN
12,True,True,True,True,True,True,True,True,True,True,...,True,True,1,True,True,True,None,None,None,None
13,2003,2003-01-06,13:10:00,Monday,"TURK ST and TAYLOR ST, 5.0 ft west",Clear,Pedestrian right of way,No,"Crossing, in crosswalk at intersection",Dry,...,Daylight,False,0,1,37.7832,-122.411,Tenderloin,012502,TENDERLOIN,TENDERLOIN
14,2003,2003-01-21,18:28:00,Tuesday,"16TH ST and CHURCH ST, at intersection",Raining,Unsafe speed,No,"Crossing, in crosswalk at intersection",Wet,...,Dark - Street lights,False,0,1,37.7645,-122.429,Castro/Upper Market,020300,MISSION,MISSION


In [13]:
fulldf[fulldf.date == True].head()

,year,date,time,dayofweek,location,WEATHER,PRIMARY_COLLISION_FACTOR,HIT_AND_RUN,PED_ACTION,ROAD_SURFACE,...,LIGHTING,ALCOHOL_INVOLVED,COUNT_PED_KILLED,COUNT_PED_INJURED,y,x,nhood,tractce10,police_district,hist_police_district
12,True,True,True,True,True,True,True,True,True,True,...,True,True,1,True,True,True,None,None,None,None
24,True,True,True,True,True,True,True,True,True,True,...,True,True,1,True,True,True,None,None,None,None
35,True,True,True,True,True,True,True,True,True,True,...,True,True,1,True,True,True,None,None,None,None
43,True,True,True,True,True,True,True,True,True,True,...,True,True,1,True,True,True,None,None,None,None
50,True,True,True,True,True,True,True,True,True,True,...,True,True,1,True,True,True,None,None,None,None


In [5]:
# Read data
fulldf = pd.read_pickle('../data/trafficcollisions.p')

lowercase = 'trafficcollisions'
uppercase = 'Traffic Collisions'

# Add month and columns
fulldf['month'] = fulldf.date.apply(lambda x: x.month)

# Read weather data
weatherfull = pd.read_pickle(r"../data/noaa-weather-downtown-sf.p")


# Convert celsius to fahrenheit
def celsius2fahrenheit(T):
    return T * 1.8 + 32
weatherfull['TMAX'] = weatherfull['TMAX'].apply(celsius2fahrenheit)
weatherfull['TMIN'] = weatherfull['TMIN'].apply(celsius2fahrenheit)

# Join with weather
print 'merge with weather'
print 'before merge', fulldf.shape
fulldf = fulldf.merge(weatherfull,how="inner",left_on="date",right_on="DATE")
print 'after merge', fulldf.shape
print ''

# Filter out 2016 incidents
df = fulldf[fulldf.year < 2016]
print 'fulldf', fulldf.shape
print 'df 2003-2015', df.shape
print ''
weather = weatherfull[weatherfull.DATE < date(2016, 1, 1)]
print 'full weather', weatherfull.shape
print 'weather 2003-2015', weather.shape
print ''


# Aggregate into daily counts
groupbydate = df.groupby(by=['date','dayofweek','month','year','PRCP','TMAX','TMIN']).size().to_frame(name='count')
groupbydate.reset_index(inplace=True)
print 'days from 2003/01/01 to 2015/12/31 :', date(2016,1,1)-date(2003,1,1)
print 'daily aggregate', groupbydate.shape

# Add zero counts for missing days
groupbydate = groupbydate[['date','dayofweek','month','year','count']].merge(weather,how="outer",left_on="date",right_on="DATE")
groupbydate['date']=groupbydate['DATE']
groupbydate.drop('DATE',axis=1,inplace=True)
print 'new daily aggregate', groupbydate.shape
groupbydate[groupbydate.isnull().any(axis=1)]

AttributeError: 'bool' object has no attribute 'month'